In [1]:
!pip install transformers
import torch
import json
import json
from pathlib import Path
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
import pickle
from transformers import AdamW,BertForQuestionAnswering
from tqdm.notebook import tqdm
import string, re

c:\anaconda\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\anaconda\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\anaconda\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = torch.load("savedModels/saved_model.pkl",map_location=torch.device('cpu'))

In [3]:
def ans(context,query):
  answer_start = torch.argmax(model(**tokenizer.encode_plus(query, context, return_tensors='pt'))[0]) 
  answer_end = torch.argmax(model(**tokenizer.encode_plus(query, context, return_tensors='pt'))[1]) + 1 
  return tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(tokenizer.encode_plus(query, context, return_tensors='pt')['input_ids'][0][answer_start:answer_end]))

In [4]:
def normalize(input):
  def remove_punc(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)

  return " ".join((re.sub(re.compile(r"\b(a|an|the)\b", re.UNICODE), " ", remove_punc(input.lower()))).split())

In [5]:
def f1_score(prediction, truth):
  pred_tokens = normalize(prediction).split()
  truth_tokens = normalize(truth).split()
  
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)
    
  if len(set(pred_tokens) & set(truth_tokens)) == 0:
    return 0
  
  return 2 * (len(set(pred_tokens) & set(truth_tokens)) / len(pred_tokens) * len(set(pred_tokens) & set(truth_tokens)) / len(truth_tokens)) / (len(set(pred_tokens) & set(truth_tokens)) / len(pred_tokens) + len(set(pred_tokens) & set(truth_tokens)) / len(truth_tokens))

In [6]:
def Predict_Ans(context,query,answer):
  prediction = ans(context,query)
  f1 = f1_score(prediction, answer)

  print(f"Prediction: {prediction}")
  print(f"F1: {f1}")

### Simple Context and Questions

In [7]:
context = "Hi! My name is Alexa and I am 21 years old. I used to live in Peristeri of Athens, but now I moved on in Kaisariani of Athens."

queries = ["How old is Alexa?",
           "Where does Alexa live now?",
           "Where Alexa used to live?"
          ]
answers = ["21",
           "Kaisariani of Athens",
           "Peristeri of Athens"
          ]

for q,a in zip(queries,answers):
  print(f"Question: {q}")
  print(f"Ground truth: {a}")
  Predict_Ans(context,q,a)

Question: How old is Alexa?
Ground truth: 21
Prediction: 21
F1: 1.0
Question: Where does Alexa live now?
Ground truth: Kaisariani of Athens
Prediction: kaisariani
F1: 0.5
Question: Where Alexa used to live?
Ground truth: Peristeri of Athens
Prediction: peristeri
F1: 0.5


### Large context

In [11]:
context = "Southern California, often abbreviated SoCal, is a geographic and cultural region that generally comprises California's southernmost 10 counties. The region is traditionally described as eight counties, based on demographics and economic ties: Imperial, Los Angeles, Orange, Riverside, San Bernardino, San Diego, Santa Barbara, and Ventura. The more extensive 10-county definition, including Kern and San Luis Obispo counties, is also used based on historical political divisions. Southern California is a major economic center for the state of California and the United States."

queries = ["What is Southern California often abbreviated as?"]
answers = ["SoCal"]
for q,a in zip(queries,answers):
  print(f"Question: {q}")
  print(f"Ground truth: {a}")
  Predict_Ans(context,q,a)

Question: What is Southern California often abbreviated as?
Ground truth: SoCal
Prediction: socal,
F1: 1.0


### Fine tuned model is working well on small and large context.

### Comparing model with BERT-large finetuned model

In [14]:
tokenizer_BERT_large = AutoTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


In [16]:
model_BERT_large = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


In [17]:
def ans_BERT_large(context,query):
   answer_start = torch.argmax(model_BERT_large(**tokenizer_BERT_large.encode_plus(query, context, return_tensors='pt'))[0]) 
   answer_end = torch.argmax(model_BERT_large(**tokenizer_BERT_large.encode_plus(query, context, return_tensors='pt'))[1]) + 1 
   return tokenizer_BERT_large.convert_tokens_to_string(tokenizer_BERT_large.convert_ids_to_tokens(tokenizer_BERT_large.encode_plus(query, context, return_tensors='pt')['input_ids'][0][answer_start:answer_end]))

In [20]:
def Predict_Ans_BERT_large(context,query,answer):
   prediction = ans_BERT_large(context,query)
   f1 = f1_score(prediction, answer)

   print(f"Prediction: {prediction}")
   print(f"F1: {f1}")

In [21]:
# Testing the same examples as above

In [22]:
context = "Hi! My name is Alexa and I am 21 years old. I used to live in Peristeri of Athens, but now I moved on in Kaisariani of Athens."

queries = ["How old is Alexa?",
           "Where does Alexa live now?",
           "Where Alexa used to live?"
          ]
answers = ["21",
           "Kaisariani of Athens",
           "Peristeri of Athens"
          ]

for q,a in zip(queries,answers):
  print(f"Question: {q}")
  print(f"Ground truth: {a}")
  Predict_Ans_BERT_large(context,q,a)

Question: How old is Alexa?
Ground truth: 21
Prediction: 21
F1: 1.0
Question: Where does Alexa live now?
Ground truth: Kaisariani of Athens
Prediction: kaisariani of athens
F1: 1.0
Question: Where Alexa used to live?
Ground truth: Peristeri of Athens
Prediction: peristeri of athens
F1: 1.0


In [23]:
context = "Southern California, often abbreviated SoCal, is a geographic and cultural region that generally comprises California's southernmost 10 counties. The region is traditionally described as eight counties, based on demographics and economic ties: Imperial, Los Angeles, Orange, Riverside, San Bernardino, San Diego, Santa Barbara, and Ventura. The more extensive 10-county definition, including Kern and San Luis Obispo counties, is also used based on historical political divisions. Southern California is a major economic center for the state of California and the United States."

queries = ["What is Southern California often abbreviated as?"]
answers = ["SoCal"]
for q,a in zip(queries,answers):
  print(f"Question: {q}")
  print(f"Ground truth: {a}")
  Predict_Ans_BERT_large(context,q,a)

Question: What is Southern California often abbreviated as?
Ground truth: SoCal
Prediction: socal
F1: 1.0


##### Both the models have performed simmilary

#### Testing with more complex sets

In [24]:
context = "Concerns regarding social media's effects on mental health have grown in recent years. Social media sites can help people connect, communicate, and develop communities, but they can also make people more prone to mental health problems like sadness and anxiety. Social comparison, cyberbullying, and the ongoing desire for approval and attention are just a few of the ways that social media use has been shown to have a negative impact on mental health."

queries = ["What are some of the ways in which social media use can negatively affect mental health?"]
answers = ["Use of social media can harm mental health in a variety of ways. This is caused by a number of variables, some of which have been identified by researchers, such as cyberbullying, social comparison, and the ongoing need for approval and attention. On social media, cyberbullying may take many different forms, from outright harassment to the dissemination of rumours and gossip. Feelings of guilt, loneliness, and even suicidal thoughts may result from this. Another element that might have a negative effect on mental health is social comparison. Users who are continually exposed to idealised depictions of other people's life may experience emotions of inadequacy, jealously, and self-doubt. Last but not least, the incessant demand for approval and attention engendered by social media can set off a cycle of stress and anxiety as users feel under pressure to put themselves out there."]
for q,a in zip(queries,answers):
  print(f"Question: {q}")
  print(f"Ground truth: {a}")
  Predict_Ans_BERT_large(context,q,a)

Question: What are some of the ways in which social media use can negatively affect mental health?
Ground truth: Use of social media can harm mental health in a variety of ways. This is caused by a number of variables, some of which have been identified by researchers, such as cyberbullying, social comparison, and the ongoing need for approval and attention. On social media, cyberbullying may take many different forms, from outright harassment to the dissemination of rumours and gossip. Feelings of guilt, loneliness, and even suicidal thoughts may result from this. Another element that might have a negative effect on mental health is social comparison. Users who are continually exposed to idealised depictions of other people's life may experience emotions of inadequacy, jealously, and self-doubt. Last but not least, the incessant demand for approval and attention engendered by social media can set off a cycle of stress and anxiety as users feel under pressure to put themselves out ther

In [25]:
context = "Concerns regarding social media's effects on mental health have grown in recent years. Social media sites can help people connect, communicate, and develop communities, but they can also make people more prone to mental health problems like sadness and anxiety. Social comparison, cyberbullying, and the ongoing desire for approval and attention are just a few of the ways that social media use has been shown to have a negative impact on mental health."

queries = ["What are some of the ways in which social media use can negatively affect mental health?"]
answers = ["Use of social media can harm mental health in a variety of ways. This is caused by a number of variables, some of which have been identified by researchers, such as cyberbullying, social comparison, and the ongoing need for approval and attention. On social media, cyberbullying may take many different forms, from outright harassment to the dissemination of rumours and gossip. Feelings of guilt, loneliness, and even suicidal thoughts may result from this. Another element that might have a negative effect on mental health is social comparison. Users who are continually exposed to idealised depictions of other people's life may experience emotions of inadequacy, jealously, and self-doubt. Last but not least, the incessant demand for approval and attention engendered by social media can set off a cycle of stress and anxiety as users feel under pressure to put themselves out there."]
for q,a in zip(queries,answers):
  print(f"Question: {q}")
  print(f"Ground truth: {a}")
  Predict_Ans(context,q,a)

Question: What are some of the ways in which social media use can negatively affect mental health?
Ground truth: Use of social media can harm mental health in a variety of ways. This is caused by a number of variables, some of which have been identified by researchers, such as cyberbullying, social comparison, and the ongoing need for approval and attention. On social media, cyberbullying may take many different forms, from outright harassment to the dissemination of rumours and gossip. Feelings of guilt, loneliness, and even suicidal thoughts may result from this. Another element that might have a negative effect on mental health is social comparison. Users who are continually exposed to idealised depictions of other people's life may experience emotions of inadequacy, jealously, and self-doubt. Last but not least, the incessant demand for approval and attention engendered by social media can set off a cycle of stress and anxiety as users feel under pressure to put themselves out ther

##### Both the fine tuned BERT base and BERT large have shown similar F-1 score.